## Module 2: Polynomial interpolation in 1d

*Material for this worksheet can be found in the AE2220-I lecture notes Chapter 3.*

### Problem statement

The process of constructing an approximating function which passes <i>exactly</i> though specified data points is called <i>interpolation</i>.  We concern ourselves with approximating the real function of one variable $f:\mathbb{R}\rightarrow\mathbb{R}$, on the interval $[a,b]$.  We assume knowledge of $n+1$ samples of $f(x)$ at <i>grid</i> locations $(x_0,\dots,x_n)$, with which to build the approximating function $\phi(x)$.  The requirement that $f$ passes through the data-points can be formulated as:
$$
f(x_i) = \phi(x_i) \quad\mbox{ for }\quad i = 0,\dots,n,
$$
known as the <i>interpolation conditions</i>.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'axes.labelsize': 18})
import numpy as np

To test the method, we choose a (known) analytic function, and sample it at the grid points.  Normal the exact function $f$ will be unknown.

In [ ]:
a, b = 0., 5.
n = 6

def f(x): 
    return np.sin(np.pi*x/2.)

xx = np.linspace(a,b,101)
plt.plot(xx, f(xx))
plt.xlabel(r'$x$'); plt.ylabel(r'$f$')

### Selection of the grid

Assume that we are free to choose the grid nodes $(x_0,\dots,x_n)$ - we may even choose them outside the interval $[a,b]$.  An apparently logical choice is uniformly spaced nodes:

In [ ]:
def grid_uniform(n): return np.linspace(a, b, n+1)

However, as we will see, uniform spacing performs badly in some situations.  In fact the interpolation error is minimized using Chebychev nodes, which on the interval $[-1,1]$ are:
$$
\xi_i = \cos\left( \frac{2i+1}{2(n+1)}\,\pi\right),\quad i=0,\ldots,n.
$$
These can be easily scaled to the interval $[a,b]$ with the linear map:
$$
x_i = (b-a) \frac{-\xi_i+1}{2} + a, \quad\mbox{for } i=0,\dots,n.
$$

In [ ]:
def grid_chebychev(n):
    xi = np.cos( np.pi/(2*(n+1)) * (2*np.linspace(1, n+1, n+1) - 1) )
    return ((-xi + 1)/2) * (b-a) + a

And, just to see how bad things can get with a poor choice, we consider randomly chosen nodes:

In [ ]:
def grid_random(n): return np.random.rand(n+1) * (b-a) + a

In [ ]:
plt.plot(grid_uniform(n),   np.ones(n+1)*2, 'ok', label='uniform')
plt.plot(grid_chebychev(n), np.ones(n+1)*1, 'or', label='chebychev')
plt.plot(grid_random(n),    np.ones(n+1)*0, 'ob', label='random')
plt.legend()
plt.ylim(-1,4)
plt.xlabel(r'$x$')

The difference between uniform and Chebychev becomes more noticable for larger $n$. The grid used in the following calculations is selected here.

In [ ]:
grid = grid_uniform(n)  # Choose the grid

### Selection of basis

The purpose of the basis is to provide a set of functions, which when added together with correct weights, will compose $\phi(x)$, which in turn approximates the function $f(x)$:
$$
\phi(x) := \sum_{i=0}^n a_i \varphi_i(x),
$$
where $a_i$ are weights to be found, and $\varphi_i(x)$ are $n+1$ basis functions.

Since we are concerned with polynomial approximants (because of Weierstrass, Theorem 3.4, and Uniqueness of Polynomial Interpolation, Theorem 3.5) our basis should be polynomial too.  However there are several choices.

One natural basis for polynomials are the <b>monomials</b>:
$$
\begin{align*}
m_0(x) &= 1 \\
m_1(x) &= x \\
m_2(x) &= x^2 \\
\vdots \quad & \quad\vdots \\
m_n(x) &= x^n,
\end{align*}
$$
It is self-evident that a sum of these basis functions can produce any polynomial of degree $n$.  Notice that this basis is independent of the grid points.

In [ ]:
def basis_monomial(x, grid):
    phi = np.zeros(len(grid))
    for i in range(len(grid)): phi[i] = x**i
    return phi

The <b>Newton basis</b> is a third choice that results in a lower-triangular interpolation matrix $A$:
$$ 
\pi_0(x) = 1,\quad \pi_k(x) = \prod_{j=0}^{k-1} (x-x_j),\quad k=1,2,\ldots,n.
\label{newtonbasis}
$$

In [ ]:
def basis_newton(x, grid):
    phi = np.ones(len(grid))
    for i, xi in enumerate(grid):
        for j in range(i):
            phi[i] *= (x - grid[j])
    return phi

The <b>Lagrange basis</b> function $l_i(x)$ takes the value $1$ at grid point $x_i$, and $0$ at all other grid points (we will check this in the next section), and has the form:
$$
l_i(x) = \prod_{j=0, j \neq i}^{n} \frac{x-x_j}{x_i-x_j},\quad i=0,\ldots,n.
$$

**Exercise 1:**

**(a) Implement a Lagrange basis in the function *basis_lagrange()* which takes the scalar $x \in \mathbb{R}$, the grid locations $(x_0,\dots,x_n)$ in *grid*.  It should return all $n+1$ Lagrange basis functions, evaluated at $x$.**

**(b) Use plot_basis to plot the basis functions $l_0(x),\dots,l_n(x)$ on the interval $[a,b]$.  Verify from your plot that your basis-functions satisfy the condition:
$$
l_i(x_j) = \delta_{ij} := \begin{cases} 1 & i = j \\ 0 & i \neq j \end{cases}.
$$**

**(c) It is easy to see that we can produce all polynomials of degree $n$ from a sum of monomials $m_i(x)$.  But can we produce all polynomials of degree $n$ as a sum of $l_i(x)$?  Hint:**

  0. **What degree are $l_i$?**
  0. **What do we know about a polynomial that passes through $n+1$ points?**  
  0. **Can we construct a sum of $l_i(x)$ that pass through any $n+1$ points?**
  
**(d) Finally: There is a strong connection to linear algebra.  The equivalent question there is: Does a particular set of vectors $\mathbf{l_i}$ span the linear vector-space of dimension $n+1$?  If it does then $\mathbf{l_i}$ is called a *basis*.  Can you make this connection more precise?  In particular what vector $\mathbf{l_i}$ corresponds to the function $l_i$?**

In [ ]:
def basis_lagrange(x, grid):
    pass ### TODO

In [ ]:
def plot_basis(basisfn):
    xx = np.linspace(a, b, 101)
    phi = np.zeros((101, n+1))
    for i,x in enumerate(xx): 
        phi[i] = basisfn(x,grid)
    for j in range(n+1):
        plt.plot(xx, phi[:,j])
    plt.plot(grid, np.zeros(grid.shape), 'ok', label='samples')
    plt.xlabel(r'$x$'); plt.ylabel(r'$l_i$')

In [ ]:
plot_basis(basis_lagrange)

### Constructing the interpolation conditions

The interpolation conditions state that $\phi(x_j) = f(x_j)$ at all mesh points $x_j$.  Substituting the definition of $\phi(x)$, this condition becomes:
$$
\sum_{i=1}^n a_i \varphi_i(x_j) = f(x_j) \quad\mbox{for all}\quad j\in\{0,\dots,n\}
$$
which can be rewritten in matrix form:
$$
A \mathbf{a} = \mathbf{f}
$$
where
$$
\begin{align}
   \mathbf{a} &=& (a_0,\dots, a_n) \\
   \mathbf{f} &=& (f(x_0), \dots, f(x_n)) \\
   A_{ij} &=& \varphi_i(x_j)
\end{align}
$$
The right-hand side (RHS) $\mathbf{f}$ can be evaluated as follows:

In [ ]:
def interpolation_rhs(grid):
    rhs = np.zeros(n+1)
    for i,x in enumerate(grid):
        rhs[i] = f(x)
    return rhs

In [ ]:
rhs = interpolation_rhs(grid)

**Exercise 2:**

**(a) Implement a function *interpolation_matrix()* which takes two arguments, a list of the grid nodes, and a function which describes the basis.  It should then return the interpolation matrix $A$.** 

**(b) Print the matrix using *print(A)*, and plot it, using *plt.imshow(A, interpolation='none')*.** 

**(c) How does the form of the matrix differ for monomial, Newton and Lagrange bases?  Why do the matrices have this structure?  The next step involves solving for the coefficients $\mathbf{a}$ - which matrix is easiest to invert?  Which is most difficult?**

In [ ]:
def interpolation_matrix(grid, basisfn):
    n = len(grid)-1
    A = np.zeros((n+1,n+1))
    pass ### TODO

The potentially expensive step (computationally speaking) is solving for the coefficients $\mathbf{a}$:

In [ ]:
aa = np.linalg.solve(A, rhs)
print(aa)

### Reconstructing the function

Ultimately the purpose is to reconstruct the function - here we just evaluate $\phi(x)$ using the basis functions and coefficients $\mathbf{a}$ found above:

In [ ]:
def reconstruct(x, grid):
    return np.sum(aa * basis_lagrange(x, grid))

In [ ]:
xx2 = np.linspace(0,1,101)
reconstruction = np.zeros(len(xx2))
for i,x in enumerate(xx): 
    reconstruction[i] = reconstruct(x, grid)
plt.plot(xx, f(xx), label='original')
plt.plot(grid, f(grid), 'ok', label='samples')
plt.plot(xx, reconstruction, label='interpolation')
plt.xlabel(r'$x$')
plt.legend(loc='lower right')

Whenever we get a numerical result like this, we should check to see if it meets our expectations.  Our basic expectations are:

* The interpolant is smooth;
* That it's a degree $n$ polynomial (check the number of extrema);
* Which passes through the sample points (easy to check).

If these are not true, something is seriously wrong.   On a higher level we have expectations related to **consistency**, **convergence** and **stability**:

* The interpolant approximates $f(x)$ increasingly accurately as $n$ is increased.

I.e. that the error diminishes as $n\rightarrow\infty$.

We can also ask: What expectations do we have regarding the influence of the choice of basis on the interpolant $\phi(x)$?  What about the choice of grid?  Do these inflence the numerical result or not?

### Error behaviour

One reasonable definition of error is:
$$
e_n(x) := f(x) - \phi(x)
$$
And from the notes we have a theorem regarding this error (Cauchy):

If $f \in C^{n+1}([a,b])$, then for any grid $X$ of $n+1$ nodes and for any $x \in [a,b]$, the interpolation error at $x$ is
$$
  e_n(f;x) := f(x) - \phi(x) = \frac{f^{(n+1)}(\xi)}{(n+1)!}\,\omega_{n+1}(x),
$$
where $\xi = \xi(x) \in (\min_k(x_k,x), \max_k(x_k,x)) \subset [a,b]$ and $\omega_{n+1}(x)$ is the <i>nodal polynomial</i> associated with the grid $\mathbf{x}$, defined as,
$$
\omega_{n+1}(x) := \prod_{i=0}^n (x-x_i).
$$

The problem with this error estimate is that it's not readily <i>computable</i>, as computing $\xi$ and then $f^{(n+1)}(\xi)$ is not easy.  As a compromise we can assume that 
$$
f^{(n+1)}(\xi) = \mathcal{O}(1)
$$
i.e. that the left-hand side doesn't depend on $n$.  This allows us to use the error estimate:
$$
\hat e_n(x) := \frac{1}{(n+1)!}\prod_{i=0}^n (x-x_i).
$$
Let's compare theory with practice:

In [ ]:
from math import factorial
def error_estimate(x):
    return np.abs(np.prod(x - grid) / factorial(len(grid)))

### Compute estimate at each plotting point
errorest = np.zeros(len(xx))
for i,x in enumerate(xx): 
    errorest[i] = error_estimate(x)

### Do the plotting
plt.plot(xx, np.abs(f(xx) - reconstruction), label='Error exact')
plt.plot(grid, 0*grid, 'ok', label='samples')
plt.plot(xx, errorest, label='Error estimate')
plt.xlabel(r'$x$'); plt.ylabel(r'Error')
plt.legend()

The form of the error is correct, but the scale is wrong - because we don't know $f^{(n+1)}(\xi)$.  On the other hand, as we increase the number of points $f^{(n+1)}(\xi)$ will stay approximately constant, so the error estimate will describe well the *rate* of reduction of the error.

**Exercise 3: A key message of this study is that convergence of polynomial interpolation depends strongly on the choice of grid and the underlying function $f(x)$.  We want to verify these statements - we look at convergence plots.**

**Start with the function below *poly_interp()*,  which does one-shot interpolation of $f$ given a *grid*, a *basis* (e.g. the function basis_lagrange), and an array of points *xx* at which to evaluate the reconstruction:**

In [ ]:
def poly_interp(xx, f, grid, basis):
    A = interpolation_matrix(grid, basis)   ### Interpolation conditions
    ff = np.array([f(xi) for xi in grid])
    aa = np.linalg.solve(A,ff)
    basisxx = np.zeros((len(xx),len(grid))) ### Basis functions at all x values in xx
    for i,x in enumerate(xx):
        basisxx[i,:] = basis(x,grid)
    return np.dot(basisxx, aa)              ### Reconstruction

**Usage is, for example:**

In [ ]:
my_grid = grid_chebychev(10)
f_prediction = poly_interp(xx, f, my_grid, basis_monomial)
plt.plot(xx, f_prediction)
plt.plot(xx, f(xx))
plt.plot(my_grid, f(my_grid), 'ok')
plt.xlabel(r'$x$'); plt.ylabel(r'$f$')

**(a) Implement a function to approximately compute the (scalar) error:
$$
\epsilon_n := \mathrm{max}_{x\in[a,b]} | e_n(x) |
$$
for a given $f$, grid and basis.**

In [ ]:
def error_max(xx, f, grid, basis):
    pass ### TODO

**(b) Make a plot of the above error $\epsilon_n$ against the number of nodes.  Use a uniform grid an $n \in \{2,\dots,20 \}$.  Use a log-scale for the error.  You should see a line with error decreasing towards zero.**

**(c) Investigate the behaviour of the interpolation error.  Does the basis-choice have an effect on the error?  Does the grid?  What is the best choice of basis/grid, in terms of minimizing the error for a given number of nodes?  [Note: Use the function below, that produces a convergence plot for a given function *f_fn*, *grid_fn* (i.e. grid_uniform, grid_random, or grid_chebychev), and a given *basis_fn*.  By calling it multiple times, several convegence plots can be compared on a single axis.]**

In [ ]:
def plot_convergence(f_fn, grid_fn, basis_fn):
    nn, linfty = range(2, 20), []
    xx = np.linspace(a, b, 1001)
    for n in nn:
        linfty.append(error_max(xx, f_fn, grid_fn(n), basis_fn))
    plt.plot(nn, np.log10(np.array(linfty)), '-o')

In [ ]:
plot_convergence(f, grid_uniform, basis_lagrange)
plot_convergence(f, grid_chebychev, basis_lagrange)
plot_convergence(f, grid_random, basis_lagrange)
plt.xlabel(r'$N$'); plt.ylabel(r'$\log_{10}(\epsilon)$')

**(d) Our interpolant should be accurate for a range of functions.  Examine the interpolants and the convergence for the following functions.  What aspects of the theory can you see reproduced in the results?**

In [ ]:
def f_poly5(x): return 0.001*x**5 + 0.02*x**3 - x  # Polynomial
def f_step(x): return np.sign(x-2)                 # Discontinuous at x=2
def f_abs(x): return np.abs(x-2)                   # Discontinuous derivative at x=2
def f_abs3(x): return np.abs((x-2)**3)             # Discontinuous 3rd derivative at x=2
def f_runge(x): return 1./(1+(x-2)**2)             # Infinitely differentiable (everywhere)
def f_gauss(x): return np.exp(-(x-2)**2/2.)        # Very similar curve to above
def f_custom(x): pass                              # Your own function  